# Profiling a Neighborhood's Taste Palate.

### IBM Data Science Specilization's Final Project on Coursera.

<h3> Table of Contents</h3>

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 2>
 
>1. <a href="#item3.1">New York</a>
    
>2. <a href="#item3.2">California</a>

</font>
</div>

## Methodology

In [2]:
#Importing all relevant libraries
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# import k-means from clustering stage
from sklearn.cluster import KMeans


!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

##!pip install geocoder
##import geocoder

# tranforming json file into a pandas dataframe library
##from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

**Gathering the data from the source and organize it into a DataFrame**

In [3]:


df_data_1 = pd.read_csv(body)
df_data_1.head()

,zip,lat,lng,city,state_id,state_name,county_name
0,601,18.18004,-66.75218,Adjuntas,PR,Puerto Rico,Adjuntas
1,602,18.36073,-67.17517,Aguada,PR,Puerto Rico,Aguada
2,603,18.45439,-67.12202,Aguadilla,PR,Puerto Rico,Aguadilla
3,606,18.16724,-66.93828,Maricao,PR,Puerto Rico,Maricao
4,610,18.29032,-67.12243,Anasco,PR,Puerto Rico,Añasco


In [4]:
#Creating master DataFrame

df_america = pd.DataFrame(df_data_1, columns = ['zip','lat','lng','state_name'])
df_america.rename(columns = {'zip':'Zip','lat': 'Latitude','lng':'Longitude','state_name':'State'}, inplace = True)
df_america.head()

,Zip,Latitude,Longitude,State
0,601,18.18004,-66.75218,Puerto Rico
1,602,18.36073,-67.17517,Puerto Rico
2,603,18.45439,-67.12202,Puerto Rico
3,606,18.16724,-66.93828,Puerto Rico
4,610,18.29032,-67.12243,Puerto Rico


<a id='item3.1'></a>
## 1. New York

In [5]:
#Creating New York Dataframe(Subset of Master Dataframe)

df_ny = df_america[df_america['State'] == 'New York'].reset_index(drop = True) #Create New York Database
df_ny = df_ny[0:80:3] #Limit DataFrame to 27 Zip codes.
df_ny.head()

,Zip,Latitude,Longitude,State
0,6390,41.27023,-71.98781,New York
3,10003,40.73180,-73.98911,New York
6,10006,40.70968,-74.01284,New York
9,10010,40.73914,-73.98255,New York
12,10013,40.72014,-74.00476,New York


**Using Foursquare to get top rated places for each zipcode**

In [7]:
#Defining a function to get location data about each venue

def getNearbyVenues(names, latitudes, longitudes, radius=250, LIMIT = 10):
    """Creating a dataframe called nerby venues with the following data:
    Postcode, Postcode_latitude, Postcode_Longitude, Venue, Venue_Latitide, Venue_Longitude, Venue_Category, Venue_ID"""
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']
        except:
            print('No venues for postcode: ', name)
         
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name'],
            v['venue']['id']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Zipcode', 
                  'Zipcode Latitude', 
                  'Zipcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category',
                  'Venue id']
    
    return(nearby_venues)

In [8]:
ny_venues = getNearbyVenues(names=df_ny['Zip'],
                                   latitudes=df_ny['Latitude'],
                                   longitudes=df_ny['Longitude']
                                  )

6390
10003
10006
10010
10013
10017
10020
10023
10026
10029
10032
10035
10038
10044
10075
10111
10119
10153
10165
10169
10172
10177
10278
10282
10303
10306
10309


In [9]:
ny_venues.shape

(235, 8)

In [10]:
#Defining a function to get the ratings based on the Venue ID

def GetRatings(names, IDs):
    ratings_list = []

    for name, ID in zip(names,IDs):
        
        url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(
            ID, 
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
            
# make the GET request
        try:
            results = requests.get(url).json()['response']['venue']['rating']
            print(name, results)
            
            ratings_list.append([(
            ID,
            requests.get(url).json()['response']['venue']['rating'])])
            
        except:
            print(name, ' Na.')
            ratings_list.append([(
            ID,
            'Na')])

        
    ratings = pd.DataFrame([item for rating_list in ratings_list for item in rating_list])
    ratings.columns = ['Venue id', 'Rating']
    print('Process Complete')              
    
    return(ratings)

In [11]:
#USE THIS THE FIRST TIME TO GET ny_ratings DATAFRAME
#ny_ratings = GetRatings(names = ny_venues['Venue'],  
#                        IDs = ny_venues['Venue id'])

# SAVE DATA TO FILE STORAGE AFTER FIRST PASS OF ABOVE FUNCTION
#project.save_data(data=ny_ratings.to_csv(index=False),file_name='ny_ratings.csv',overwrite=True)

#IMPORT NY RATINGS DATAFRAME FROM FILE
#HIDDEN DATA

ny_ratings = pd.read_csv(body)
ny_ratings.head()


,Venue id,Rating
0,4b6cba70f964a520ae4f2ce3,9.4
1,51351c3fe4b0c58b63965114,9.1
2,5831db480162176af6ed9c15,9.1
3,4a5403b8f964a520f3b21fe3,9.1
4,4c534e8930f92d7ffedc8db8,9.1


**SAVE RATINGS DATA TO PROJECT**

Due to limited no of avaliable API calls to foursquare

In [12]:
# Save dataframe as csv file to storage
#project.save_data(data=ny_ratings.to_csv(index=False),file_name='ny_ratings.csv',overwrite=True)

In [13]:
#Creating DataFrame with all New York related data

ny_complete = pd.merge(ny_venues, ny_ratings, on = 'Venue id')
ny_complete.head()

,Zipcode,Zipcode Latitude,Zipcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue id,Rating
0,10003,40.7318,-73.98911,Peridance Capezio Center,40.732987,-73.988522,Dance Studio,4b6cba70f964a520ae4f2ce3,9.4
1,10003,40.7318,-73.98911,Joe's Pizza,40.733234,-73.987672,Pizza Place,51351c3fe4b0c58b63965114,9.1
2,10003,40.7318,-73.98911,Switch Playground 12th Street,40.732184,-73.988699,Gym,5831db480162176af6ed9c15,9.1
3,10003,40.7318,-73.98911,Ippudo,40.730948,-73.990287,Ramen Restaurant,4a5403b8f964a520f3b21fe3,9.1
4,10003,40.7318,-73.98911,Fabio Clemente Jiu Jitsu,40.732304,-73.989069,Martial Arts Dojo,4c534e8930f92d7ffedc8db8,9.1


In [14]:
#Creating DataFrame with columns needed to further sort

ny_group = pd.DataFrame(ny_complete, columns = ['Zipcode','Zipcode Latitude','Zipcode Longitude','Venue Category','Rating'])
ny_group = ny_group[ny_group['Rating'] != 'Na']
ny_group.head()

,Zipcode,Zipcode Latitude,Zipcode Longitude,Venue Category,Rating
0,10003,40.7318,-73.98911,Dance Studio,9.4
1,10003,40.7318,-73.98911,Pizza Place,9.1
2,10003,40.7318,-73.98911,Gym,9.1
3,10003,40.7318,-73.98911,Ramen Restaurant,9.1
4,10003,40.7318,-73.98911,Martial Arts Dojo,9.1


**Creating One-hot Vector Matrix**

In [15]:
#Create a one-hot vector
ny_onehot = pd.get_dummies(ny_group[['Venue Category']], prefix="", prefix_sep="") 
ny_onehot

,African Restaurant,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,Bagel Shop,Bakery,Bank,...,Tea Room,Thai Restaurant,Theater,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Warehouse Store,Wine Bar,Wine Shop,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
#Creating onehot Total Matrix

#Onehot vector * Ratings = matrix with all the ratings
ny_onehot_T = ny_onehot.multiply(ny_group['Rating'].astype(float), axis = 0)

# add Zipcode column back to dataframe
ny_onehot_T['Zipcode'] = ny_group['Zipcode'] 

# move neighborhood column to the first column
fixed_columns = [ny_onehot_T.columns[-1]] + list(ny_onehot_T.columns[:-1])
ny_onehot_T = ny_onehot_T[fixed_columns]

ny_onehot_T = ny_onehot_T.groupby('Zipcode').sum().reset_index()

ny_onehot_T

,Zipcode,African Restaurant,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,Bagel Shop,Bakery,...,Tea Room,Thai Restaurant,Theater,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Warehouse Store,Wine Bar,Wine Shop,Yoga Studio
0,10003,0.0,0.0,0.0,0.0,0.0,0.0,8.9,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.6,0.0
2,10010,0.0,0.0,8.9,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.4
3,10013,0.0,0.0,0.0,0.0,8.9,0.0,0.0,0.0,0.0,...,0.0,0.0,8.8,0.0,0.0,0.0,0.0,9.0,0.0,0.0
4,10017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,10020,0.0,18.2,0.0,18.4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0
6,10023,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.8,0.0,8.5
7,10026,16.6,0.0,0.0,0.0,0.0,0.0,0.0,8.9,7.5,...,0.0,0.0,0.0,0.0,8.3,0.0,0.0,0.0,0.0,0.0
8,10029,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,7.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,10032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.9,...,0.0,7.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
#Creating NOI(number of instances) onehot matrix

#Onehot vector.sum() = matrix with number of instances of each venue per zipcode
ny_onehot_N = ny_onehot

# add Zipcode column back to dataframe
ny_onehot_N['Zipcode'] = ny_group['Zipcode'] 

# move neighborhood column to the first column
fixed_columns = [ny_onehot_N.columns[-1]] + list(ny_onehot_N.columns[:-1])
ny_onehot_N = ny_onehot_N[fixed_columns]

ny_onehot_N = ny_onehot_N.groupby('Zipcode').sum().reset_index()

ny_onehot_N

,Zipcode,African Restaurant,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,Bagel Shop,Bakery,...,Tea Room,Thai Restaurant,Theater,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Warehouse Store,Wine Bar,Wine Shop,Yoga Studio
0,10003,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10006,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,10010,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,10013,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
4,10017,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,10020,0,2,0,2,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
6,10023,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,1
7,10026,2,0,0,0,0,0,0,1,1,...,0,0,0,0,1,0,0,0,0,0
8,10029,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
9,10032,0,0,0,0,0,0,0,0,2,...,0,1,0,0,0,0,0,0,0,0


In [18]:
#Creating matrix with average rating of each venue per Zipcode

ny_onehot_average = ny_onehot_T / ny_onehot_N

#Remove redundant Zipcode Column
ny_onehot_average.drop(columns='Zipcode', inplace = True)

# add Zipcode column back to dataframe
ny_onehot_average['Zipcode'] = ny_onehot_N['Zipcode'] 

# move neighborhood column to the first column
fixed_columns = [ny_onehot_average.columns[-1]] + list(ny_onehot_average.columns[:-1])
ny_onehot_average = ny_onehot_average[fixed_columns]

#Fill NaN values with 0
ny_onehot_average.fillna(0, inplace = True)
ny_onehot_average

,Zipcode,African Restaurant,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,Bagel Shop,Bakery,...,Tea Room,Thai Restaurant,Theater,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Warehouse Store,Wine Bar,Wine Shop,Yoga Studio
0,10003,0.0,0.0,0.0,0.0,0.0,0.0,8.9,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.6,0.0
2,10010,0.0,0.0,8.9,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.4
3,10013,0.0,0.0,0.0,0.0,8.9,0.0,0.0,0.0,0.00,...,0.0,0.0,8.8,0.0,0.0,0.0,0.0,9.0,0.0,0.0
4,10017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,10020,0.0,9.1,0.0,9.2,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0
6,10023,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.80,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.8,0.0,8.5
7,10026,8.3,0.0,0.0,0.0,0.0,0.0,0.0,8.9,7.50,...,0.0,0.0,0.0,0.0,8.3,0.0,0.0,0.0,0.0,0.0
8,10029,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,7.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,10032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.95,...,0.0,7.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Creating DataFrame with top 3 Venue Categories for each Zipcode**

In [19]:
#Sort the Venues in Decending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [20]:
#Create dataframe with top venues
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Zipcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Best Venue Category'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Best Venue Category'.format(ind+1))

# create a new dataframe
ny_venues_sorted = pd.DataFrame(columns=columns)
ny_venues_sorted['Zipcode'] = ny_onehot_average['Zipcode']

for ind in np.arange(ny_onehot_average.shape[0]):
    ny_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_onehot_average.iloc[ind, :], num_top_venues)

ny_venues_sorted.head()

,Zipcode,1st Best Venue Category,2nd Best Venue Category,3rd Best Venue Category
0,10003,Dance Studio,Ramen Restaurant,Gym
1,10006,Park,Gourmet Shop,Memorial Site
2,10010,Hotel,Arcade,Cocktail Bar
3,10013,Lounge,Wine Bar,Café
4,10017,Martial Arts Dojo,Sake Bar,Japanese Restaurant


**Clustering Data**

In [21]:
### CLUSTERTING

# set number of clusters
kclusters = 6

ny_clustering = ny_onehot_average.drop('Zipcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters).fit(ny_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:]

array([5, 0, 0, 1, 2, 4, 3, 3, 0, 0, 0, 1, 1, 3, 4, 3, 1, 1, 2, 2, 2, 3,
       0], dtype=int32)

In [22]:
### ADDING LABELS

# add clustering labels
try:
    ny_venues_sorted.drop(columns = 'Cluster Labels', inplace = True)
except:
    print('No column yet')
    
ny_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

No column yet


**Vizulization using Folium**

In [23]:
#Create DataFrame with Zip, lat and lng
ny_data = pd.DataFrame(ny_group, columns = ['Zipcode','Zipcode Latitude','Zipcode Longitude']).drop_duplicates(subset = 'Zipcode', keep = 'first')

#Creating complete DataFrame
ny_data = ny_data.join(ny_venues_sorted.set_index('Zipcode'), on='Zipcode')
ny_data.head()

,Zipcode,Zipcode Latitude,Zipcode Longitude,Cluster Labels,1st Best Venue Category,2nd Best Venue Category,3rd Best Venue Category
0,10003,40.73180,-73.98911,5,Dance Studio,Ramen Restaurant,Gym
10,10006,40.70968,-74.01284,0,Park,Gourmet Shop,Memorial Site
20,10010,40.73914,-73.98255,0,Hotel,Arcade,Cocktail Bar
29,10013,40.72014,-74.00476,1,Lounge,Wine Bar,Café
39,10017,40.75230,-73.97249,2,Martial Arts Dojo,Sake Bar,Japanese Restaurant


In [24]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York are 40.7896239, -73.9598939.


In [25]:
### VISUALIZATION

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
#colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
colors_array = cm.tab10(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_data['Zipcode Latitude'], ny_data['Zipcode Longitude'], ny_data['Zipcode'], ny_data['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color= rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [56]:
ny_data.loc[ny_data['Cluster Labels'] == 0, ny_data.columns[[0]+[4] + list(range(5, ny_data.shape[1]))]]

,Zipcode,1st Best Venue Category,2nd Best Venue Category,3rd Best Venue Category
10,10006,Park,Gourmet Shop,Memorial Site
20,10010,Hotel,Arcade,Cocktail Bar
122,10029,Beer Bar,Mexican Restaurant,Park
132,10032,Park,Mexican Restaurant,Bakery
142,10035,Warehouse Store,Pet Store,Video Game Store
269,10282,Athletics & Sports,Dog Run,Park


In [55]:
ny_data.loc[ny_data['Cluster Labels'] == 1, ny_data.columns[[0]+[4] + list(range(5, ny_data.shape[1]))]]

,Zipcode,1st Best Venue Category,2nd Best Venue Category,3rd Best Venue Category
29,10013,Lounge,Wine Bar,Café
152,10038,Paper / Office Supplies Store,Ice Cream Shop,Nail Salon
161,10044,Outdoors & Recreation,Greek Restaurant,Sandwich Place
192,10153,Tea Room,Hotel,Jewelry Store
201,10165,Smoke Shop,Sandwich Place,Train Station


<a id='item3.2'></a>
## 2. California

In [26]:
#Creating California Dataframe(Subset of Master Dataframe)

df_ca = df_america[df_america['State'] == 'California'].reset_index(drop = True) #Create Los Angeles Database
df_ca = df_ca[0:90:3] #Limit DataFrame to 30 Zip codes.
df_ca.head()

,Zip,Latitude,Longitude,State
0,90001,33.97397,-118.24953,California
3,90004,34.07621,-118.31084,California
6,90007,34.02809,-118.28489,California
9,90011,34.00714,-118.25874,California
12,90014,34.04308,-118.25169,California


**Using FourSquare to get Venue Data**

In [27]:
#Getting DataFrame of Venues of California

ca_venues = getNearbyVenues(names=df_ca['Zip'],
                                   latitudes=df_ca['Latitude'],
                                   longitudes=df_ca['Longitude']
                                  )

90001
90004
90007
90011
90014
90017
90020
90023
90026
90029
90033
90036
90039
90042
90045
90048
90057
90061
90064
90067
90071
90079
90094
90210
90220
90230
90241
90247
90250
90260


**Using FourSquare to get Ratings for each Venue**

In [28]:
#Getting DataFrame of Ratings of Venues in California
#"""Use first time to get venue ratings. Save this dataframe for easy access to data later. This is done due to FourSquare having a very limited number of API calls per day"""
#ca_ratings = GetRatings(names = ca_venues['Venue'], 
#                        IDs = ca_venues['Venue id'])

#SAVE DATA TO FILE STORAGE AFTER FIRST PASS OF ABOVE FUNCTION
#project.save_data(data=ca_ratings.to_csv(index=False),file_name='ca_ratings.csv',overwrite=True)

#LOAD DATA FROM FILE
#HIDDEN DATA

ca_ratings = pd.read_csv(body)
ca_ratings.head()


,Venue id,Rating
0,4d9e0b3a8efaa14314c274b7,7.0
1,4b6d150ef964a52011662ce3,6.5
2,4c781f633badb1f78bb74a54,6.5
3,599f1997e97dfb71b71a342a,6.5
4,4c6000ec3a3703bb5c1ae406,6.4


**Creating DataFrame with California Data**

In [29]:
#Creating DataFrame with all California related data

ca_complete = pd.merge(ca_venues, ca_ratings, on = 'Venue id')
ca_complete.head()

,Zipcode,Zipcode Latitude,Zipcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue id,Rating
0,90001,33.97397,-118.24953,Superior Grocers,33.973280,-118.247079,Grocery Store,4d9e0b3a8efaa14314c274b7,7.0
1,90001,33.97397,-118.24953,Jack in the Box,33.975167,-118.250313,Fast Food Restaurant,4b6d150ef964a52011662ce3,6.5
2,90001,33.97397,-118.24953,SUBWAY,33.975231,-118.248249,Sandwich Place,4c781f633badb1f78bb74a54,6.5
3,90001,33.97397,-118.24953,Family Dollar,33.974478,-118.248578,Discount Store,599f1997e97dfb71b71a342a,6.5
4,90001,33.97397,-118.24953,Pizza Hut,33.975231,-118.248249,Pizza Place,4c6000ec3a3703bb5c1ae406,6.4


In [30]:
#Creating DataFrame with columns needed to further sort

ca_group = pd.DataFrame(ca_complete, columns = ['Zipcode','Zipcode Latitude','Zipcode Longitude','Venue Category','Rating'])
ca_group = ca_group[ca_group['Rating'] != 'Na']
ca_group.head()

,Zipcode,Zipcode Latitude,Zipcode Longitude,Venue Category,Rating
0,90001,33.97397,-118.24953,Grocery Store,7.0
1,90001,33.97397,-118.24953,Fast Food Restaurant,6.5
2,90001,33.97397,-118.24953,Sandwich Place,6.5
3,90001,33.97397,-118.24953,Discount Store,6.5
4,90001,33.97397,-118.24953,Pizza Place,6.4


**Creating One-hot Vector Matrix**

In [31]:
#Create a one-hot vector
ca_onehot = pd.get_dummies(ca_group[['Venue Category']], prefix="", prefix_sep="") 

In [32]:
#Creating onehot Total Matrix

#Onehot vector * Ratings = matrix with all the ratings
ca_onehot_T = ca_onehot.multiply(ca_group['Rating'].astype('float'), axis = 0)

# add Zipcode column back to dataframe
ca_onehot_T['Zipcode'] = ca_group['Zipcode'] 

# move neighborhood column to the first column
fixed_columns = [ca_onehot_T.columns[-1]] + list(ca_onehot_T.columns[:-1])
ca_onehot_T = ca_onehot_T[fixed_columns]

ca_onehot_T = ca_onehot_T.groupby('Zipcode').sum().reset_index()

ca_onehot_T.head()

,Zipcode,ATM,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bank,Bar,...,Shoe Repair,Shopping Mall,Steakhouse,Sushi Restaurant,Taco Place,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,90001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,90004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,...,6.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,90007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,90011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,90014,0.0,0.0,0.0,8.4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,8.3


In [33]:
#Creating NOI(number of instances) onehot matrix

#Onehot vector.sum() = matrix with number of instances of each venue per zipcode
ca_onehot_N = ca_onehot

# add Zipcode column back to dataframe
ca_onehot_N['Zipcode'] = ca_group['Zipcode'] 

# move neighborhood column to the first column
fixed_columns = [ca_onehot_N.columns[-1]] + list(ca_onehot_N.columns[:-1])
ca_onehot_N = ca_onehot_N[fixed_columns]

ca_onehot_N = ca_onehot_N.groupby('Zipcode').sum().reset_index()

ca_onehot_N.head()

,Zipcode,ATM,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bank,Bar,...,Shoe Repair,Shopping Mall,Steakhouse,Sushi Restaurant,Taco Place,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,90001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,90004,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
2,90007,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,90011,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,90014,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1


In [34]:
#Creating matrix with average rating of each venue per Zipcode

ca_onehot_average = ca_onehot_T / ca_onehot_N

#Remove redundant Zipcode Column
ca_onehot_average.drop(columns='Zipcode', inplace = True)

# add Zipcode column back to dataframe
ca_onehot_average['Zipcode'] = ca_onehot_N['Zipcode'] 

# move neighborhood column to the first column
fixed_columns = [ca_onehot_average.columns[-1]] + list(ca_onehot_average.columns[:-1])
ca_onehot_average = ca_onehot_average[fixed_columns]

#Fill NaN values with 0
ca_onehot_average.fillna(0, inplace = True)
ca_onehot_average

,Zipcode,ATM,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bank,Bar,...,Shoe Repair,Shopping Mall,Steakhouse,Sushi Restaurant,Taco Place,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,90001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
1,90004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,...,6.7,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
2,90007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,90011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,90014,0.0,0.0,0.0,8.4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,9.0,0.0,0.00,0.0,0.0,8.3
5,90017,0.0,8.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
6,90020,0.0,0.0,0.0,0.0,8.1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,8.9,0.0
7,90026,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7.3,0.00,8.4,0.0,0.0
8,90029,0.0,0.0,0.0,0.0,0.0,0.0,7.2,0.0,0.0,...,0.0,0.0,0.0,0.0,6.3,0.0,0.00,0.0,0.0,0.0
9,90033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


**Creating DataFrame with top 3 Venue Categories for each Zipcode**

In [35]:
#Create dataframe with top venues
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Zipcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
ca_venues_sorted = pd.DataFrame(columns=columns)
ca_venues_sorted['Zipcode'] = ca_onehot_average['Zipcode']

for ind in np.arange(ca_onehot_average.shape[0]):
    ca_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ca_onehot_average.iloc[ind, :], num_top_venues)

ca_venues_sorted.head()

,Zipcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,90001,Grocery Store,Fast Food Restaurant,Sandwich Place
1,90004,Salon / Barbershop,Health & Beauty Service,Fried Chicken Joint
2,90007,Gastropub,Caribbean Restaurant,Yoga Studio
3,90011,Discount Store,Yoga Studio,Coworking Space
4,90014,Salon / Barbershop,Taco Place,Hotel


**Clustering Data**

In [36]:
### CLUSTERTING

# set number of clusters
kclusters = 5

ca_clustering = ca_onehot_average.drop('Zipcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters).fit(ca_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:]

array([1, 1, 1, 1, 0, 3, 1, 1, 1, 1, 0, 2, 1, 1, 3, 3, 0, 4, 1, 1, 1, 1],
      dtype=int32)

In [37]:
### ADDING LABELS

# add clustering labels
try:
    ca_venues_sorted.drop(columns = 'Cluster Labels', inplace = True)
except:
    print('No column yet')
    
ca_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

No column yet


**Visualization**

In [38]:
#Create DataFrame with Zip, lat and lng
ca_data = pd.DataFrame(ca_group, columns = ['Zipcode','Zipcode Latitude','Zipcode Longitude']).drop_duplicates(subset = 'Zipcode', keep = 'first')

#Creating complete DataFrame
ca_data = ca_data.join(ca_venues_sorted.set_index('Zipcode'), on='Zipcode')
ca_data.head()

,Zipcode,Zipcode Latitude,Zipcode Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,90001,33.97397,-118.24953,1,Grocery Store,Fast Food Restaurant,Sandwich Place
8,90004,34.07621,-118.31084,1,Salon / Barbershop,Health & Beauty Service,Fried Chicken Joint
18,90007,34.02809,-118.28489,1,Gastropub,Caribbean Restaurant,Yoga Studio
20,90011,34.00714,-118.25874,1,Discount Store,Yoga Studio,Coworking Space
21,90014,34.04308,-118.25169,0,Salon / Barbershop,Taco Place,Hotel


In [39]:
address = 'Los Angeles, California, USA'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of California are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of California are 34.0536909, -118.2427666.


In [40]:
### VISUALIZATION

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
#colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
colors_array = cm.tab10(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ca_data['Zipcode Latitude'], ca_data['Zipcode Longitude'], ca_data['Zipcode'], ca_data['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color= rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Checking Clusters**

In [58]:
ca_data.loc[ca_data['Cluster Labels'] == 1, ca_data.columns[[0]+[4] + list(range(5, ca_data.shape[1]))]]

,Zipcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,90001,Grocery Store,Fast Food Restaurant,Sandwich Place
8,90004,Salon / Barbershop,Health & Beauty Service,Fried Chicken Joint
18,90007,Gastropub,Caribbean Restaurant,Yoga Studio
20,90011,Discount Store,Yoga Studio,Coworking Space
41,90020,Vietnamese Restaurant,Ice Cream Shop,Burger Joint
51,90026,Gastropub,Vegetarian / Vegan Restaurant,Food Truck
61,90029,Middle Eastern Restaurant,Donut Shop,Bakery
66,90033,Pizza Place,Yoga Studio,Fast Food Restaurant
84,90057,Hotel,Performing Arts Venue,Yoga Studio
86,90061,Park,Yoga Studio,Coworking Space
